In [1]:
import glob, os
from DecisionTrees import *
import pandas as pd
def loadimdb_train(train=True):
    if(train == True):
        pathneg= "./aclImdb/train/neg/"
        pathpos="./aclImdb/train/pos/"
    else:
        pathneg= "./aclImdb/test/neg/"
        pathpos="./aclImdb/test/pos/"
    negfiles=[]
    posfiles=[]
    for file in glob.glob(pathneg+"*.txt"):
        f = open(file, "r")
        negfiles.append(f.read())
    for file in glob.glob(pathpos+"*.txt"):
        f = open(file, "r")
        posfiles.append(f.read())
        
    neg=list(zip(negfiles,[0]*len(negfiles)))
    pos=list(zip(posfiles,[1]*len(posfiles)))
    sentiments = neg + pos
    return(sentiments)


In [2]:
sentiments = loadimdb_train()
testsentiments = loadimdb_train(train=False)

In [3]:
l = pd.DataFrame(sentiments)
ltest = pd.DataFrame(testsentiments)

In [4]:
data.data = l[0]
data.target=l[1]
testdata.data=ltest[0]
testdata.target=ltest[1]

In [5]:
data.target_names = ['pos','neg']
testdata.target_names=['pos','neg']

In [101]:
report(dt.cv_results_,5)

Model with rank: 1
Mean validation score: 0.734 (std: 0.009)
Parameters: {'tfidf__norm': 'l1', 'clf__splitter': 'random', 'clf__min_samples_split': 100, 'clf__min_samples_leaf': 5, 'clf__max_leaf_nodes': 50, 'clf__max_features': None, 'clf__max_depth': 100, 'clf__class_weight': 'balanced'}

Model with rank: 2
Mean validation score: 0.723 (std: 0.009)
Parameters: {'tfidf__norm': 'l1', 'clf__splitter': 'best', 'clf__min_samples_split': 100, 'clf__min_samples_leaf': 100, 'clf__max_leaf_nodes': 200, 'clf__max_features': None, 'clf__max_depth': 50, 'clf__class_weight': 'balanced'}

Model with rank: 3
Mean validation score: 0.721 (std: 0.013)
Parameters: {'tfidf__norm': 'l1', 'clf__splitter': 'best', 'clf__min_samples_split': 2, 'clf__min_samples_leaf': 50, 'clf__max_leaf_nodes': 20, 'clf__max_features': None, 'clf__max_depth': None, 'clf__class_weight': None}

Model with rank: 4
Mean validation score: 0.717 (std: 0.008)
Parameters: {'tfidf__norm': 'l1', 'clf__splitter': 'best', 'clf__min_sa

In [103]:
report(bestada.cv_results_,10)

Model with rank: 1
Mean validation score: 0.851 (std: 0.002)
Parameters: {'clf__n_estimators': 600, 'clf__learning_rate': 1.5, 'clf__base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best'), 'clf__algorithm': 'SAMME'}

Model with rank: 2
Mean validation score: 0.847 (std: 0.001)
Parameters: {'clf__n_estimators': 600, 'clf__learning_rate': 0.9, 'clf__base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_spl

In [72]:
dt=run_pipeline(data)
bestada = run_pipeline(data,AdaBoostClassifier(random_state=0),
                      params={
                             'clf__base_estimator':[DecisionTreeClassifier(max_depth=2)],
                             'clf__n_estimators': [10,100,200,600],
                             'clf__learning_rate':[.5,.9,1,1.5],
                             'clf__algorithm':['SAMME.R','SAMME'],
                             })

/Users/Rgao/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/Rgao/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


SearchCV took 16378.64 seconds for 10 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.851 (std: 0.002)
Parameters: {'clf__n_estimators': 600, 'clf__learning_rate': 1.5, 'clf__base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best'), 'clf__algorithm': 'SAMME'}

Model with rank: 2
Mean validation score: 0.847 (std: 0.001)
Parameters: {'clf__n_estimators': 600, 'clf__learning_rate': 0.9, 'clf__base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_s

In [6]:
import sklearn
from sklearn import datasets
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from time import time
import numpy as np
def load_data(data, input_features, target_feature, cat_features=None, num_features=None, txt_features=None):
    all_features = input_features + [target_feature]
    print(data.values.shape)
    data = data[all_features]
    data.dropna(subset=[target_feature], inplace=True)

    # change categorical features to numeric code
    if(cat_features!=None):
        data[cat_features] = data[cat_features].astype('category')
        data[cat_features] = data[cat_features].apply(lambda x: x.cat.codes)
    # replace nan with 0 in numerical features
    if(num_features!=None):
        data[num_features] = data[num_features].fillna(0.)
        for feature in num_features:
            data[feature] = data[feature].apply(lambda x: replace_string(x))
    if(txt_features!=None):
        if txt_features:
            data[txt_features] = data[txt_features].fillna('')

    return data
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            
from sklearn.pipeline import Pipeline
def build_pipeline(model = DecisionTreeClassifier(random_state=0, criterion='gini')):
    return Pipeline(([
        ('vect',CountVectorizer(stop_words='english')),
        ('tfidf',TfidfTransformer(use_idf= True)),
        ('clf',model),
    ]))
def run_pipeline(text_ds,model=DecisionTreeClassifier(random_state=0, criterion='gini'),
                 gridsearch =False,
                 params ={'clf__splitter':['best','random'],
                          'tfidf__norm':['l2','l1'],
                                    'clf__max_features':["auto","sqrt","log2",None],
                                    'clf__class_weight': ['balanced',None],
                                    'clf__min_samples_leaf':[1,5,20,50,100],
                                    'clf__min_samples_split':[2,5,20,50,100],
                                    'clf__max_depth':[25,50,100,None],
                                    'clf__max_leaf_nodes':[20,50,200,None]},
                ):
    pl = build_pipeline(model)
    pl.fit(text_ds.data,text_ds.target)
    
    if(gridsearch==True):
        search = GridSearchCV(pl, params, n_jobs=4,verbose=1)
    else:
        search =  RandomizedSearchCV(pl, param_distributions=params,
                                   n_iter=20)
    start = time()
    search.fit(text_ds.data,text_ds.target)
    search.fit(data.data, data.target)
    print("SearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), 20))
    report(search.cv_results_)
    return(search)




In [7]:

ba=build_pipeline(AdaBoostClassifier(n_estimators=600,learning_rate=1.5,base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2),algorithm='SAMME'))
ba.fit(data.data,data.target)
bd=build_pipeline(DecisionTreeClassifier(class_weight='balanced',criterion='gini',max_depth=100,max_features=None,max_leaf_nodes=50,min_samples_leaf=5,splitter='random',min_samples_split=100))
bd.fit(data.data,data.target)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabular...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 DecisionTreeClassifier(class_weight='balanced',
                                        criterion='gini', max_depth=100,
                                        ma

In [8]:
dtpred = bd.predict(testdata.data)
print(accuracy_score(dtpred, testdata.target))
adapred= ba.predict(testdata.data)
print(accuracy_score(adapred, testdata.target))
dtcv=cross_validate(plaindt,data.data,data.target,3)
adacv=cross_validate(plainada,data.data,data.target,3)

0.7578
0.85104


NameError: name 'cross_validate' is not defined

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
def cross_validate(model,X_input,target,cv):
    return(cross_val_score(model, X_input, target, cv=cv))

In [73]:
plaindt = build_pipeline(DecisionTreeClassifier())
plainada = build_pipeline(AdaBoostClassifier())
plaindt.fit(data.data,data.target)
plainada.fit(data.data,data.target)

dtpred = plaindt.predict(testdata.data)
print(accuracy_score(dtpred, testdata.target))
adapred= plainada.predict(testdata.data)
print(accuracy_score(adapred, testdata.target))
dtcv=cross_validate(plaindt,data.data,data.target,3)
adacv=cross_validate(plainada,data.data,data.target,3)


0.70348
0.8034


array([0.79841613, 0.8012959 , 0.79848776])

In [96]:
print(confusion_matrix(dtpred,testdata.target))
print(confusion_matrix(adapred,testdata.target))

[[8833 3746]
 [3667 8754]]
[[ 9564  1979]
 [ 2936 10521]]


In [93]:
print(np.mean(dtcv))
print(np.std(dtcv))
print(np.mean(adacv))
print(np.std(adacv))
print(confusion_matrix(dtpred,testdata.target))
print(confusion_matrix(adapred,testdata.target))

NameError: name 'dtcv' is not defined

In [ ]:
badapred=bestada.predict(testdata.data)
print(accuracy_score(badapred, testdata.target))
bdtpred = dt.predict(testdata.data)
print(accuracy_score(bdtpred, testdata.target))
bestadacv=cross_validate(bestada,data.data,data.target,3)
bdtcv=cross_validate(dt,data.data,data.target,3)

print(np.mean(bdtcv))
print(np.std(bdtcv))
print(np.mean(bestadacv))
print(np.std(bestadacv))

print(confusion_matrix(bdtpred,testdata.target))
print(confusion_matrix(badapred,testdata.target))

In [91]:
dt.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 TfidfTransformer(norm='l1', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 DecisionTreeClassifier(class_weight='balanced',
                                        criterion='gini', max_depth=100,
                                        ma

In [95]:
bestada.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                                    base_estimator=DecisionTreeClassifier(class_weight=None,
                                                                          criterion='gini',
                                                                          max_depth=2,
                                                                